# Quantum Teleportation
------

Alice has a qubit in state $
\left|\psi\right>=\alpha\left|0\right>+\beta\left|1\right>$.

She wants to send this quantum information to Bob.
However the laws of quantum mechanics prevent making a copy of her qubit (the no-cloning theorem). Alice can transfer her state to Bob using two classical bits and an entangled pair. This is 'teleportation' because the original qubit is not physically transferred but afterwards Bob will have state $\left|\psi\right>$ and Alice will not have it anymore.

------

### Overview

One qubit of an entangled pair is given to Alice and one is given to Bob.

Alice measures her two qubits in the Bell basis and sends the (classical) results to Bob.

Using those classical bits, Bob will decide whether to apply particular gates to his qubit.

Afterwards Bob will have the state $\left|\psi\right>$.

------

First we import `pennylane` and initialize a quantum device.

In [4]:
# imports
import pennylane as qml
from pennylane import numpy as np

# first we set up device
dev = qml.device("qiskit.aer", wires=3, shots=1000)
# we will use the qiskit simulator so we can make prettier circuit diagrams

The following function will be used to create the initial state $\left|\psi\right>$.

In [ ]:
def initialize_psi(theta,phi,wire=0):
    qml.Rot(0,theta,phi,wires=wire)

> ***Exercise 1.***
>Create a Bell pair on qubits 1 and 2.
>(i.e. the state $\frac{1}{\sqrt{2}}\left|00\right>+\left|11\right>$)
> by completing the following function.

In [ ]:
def create_Bell_state():
    # Your solution here

Let's pretend qubit `1` is sent to Alice and qubit `2` is sent to Bob (on a physical quantum computer the bits cannot be sent anywhere).

Therefore Alice can only manipulate qubits `0` and `1`. She will apply the following gates.

In [ ]:
def Alice_gates():
    qml.CNOT(wires=[0,1])
    qml.Hadamard(wires=0)

> ***Exercise 2.***
>Show that, if Alice's qubits are measured after applying her gates, they will  in the Bell basis.
>i.e. Show that the quantum 
circuit 
><div><img src="images/bell.jpg" width="200"></div> 
maps the four Bell states 
>$$
\begin{aligned}
    \left|\Phi_+ \right> &= \frac{1}{\sqrt{2}} \left( \left|00 \right> + \left|11 \right> \right)\\
    \left|\Phi_- \right> &= \frac{1}{\sqrt{2}} \left( \left|00 \right> - \left|11 \right> \right)\\
    \left|\Psi_+ \right> &= \frac{1}{\sqrt{2}} \left( \left|01 \right> + \left|10 \right> \right)\\
    \left|\Psi_+ \right> &= \frac{1}{\sqrt{2}} \left( \left|01 \right> - \left|10 \right> \right) \,.
\end{aligned}
$$
>to the computational basis states: 
$\left|00\right>, \left|01\right>,\left|10\right>, \left|11\right>$ 

Teleportation can be achieved if Alice measures her qubits (`0` and `1`) and sends the results (classical bits) to Bob.
<div><img src="images/bell_measure.jpg" width="200"></div> 

Bob will then manipulate his qubit (`2`) based on those results.

Depending on the measurement outcomes, Bob needs to make some adjustments to qubit `2` as shown below. Here, $Z^{M_1}$ and $X^{M_2}$ means that the Pauli gates are only applied when $M_1=1$ and $M_2=1$, respectively. These single-qubit gates are sometimes referred to as *Pauli corrections*.

This circuit constitutes are final teleportation algorithm:

<div>
<img src="images/tele_circuit.jpg" width="700">
</div>

Alternate version of circuit diagram (via `qiskit`, with an example):
<div>
<img src="images/qiskit_teleportation.png" width="700">
</div>

In `pennylane` and on current quantum hardware we are limited to performing all measurements at the end. So we cannot in practice manipulate qubit 2 after measuring 0 and 1.

However we can still produce $\left|\psi\right>$ on qubit 2
due to the *deferred measurement principle* [1].
This principle states that any measurement can be postponed until the end of the circuit, i.e. we can move all the measurements to the end, and we should see the same results.

The measurement controlled Pauli gates can be converted to regular controlled gates.

<div>
<img src="images/defer_measurement.svg" width="300">
</div>

In [ ]:
# instead of measuring at this point, we apply controlled gates
def Bob_gates():
    qml.CNOT(wires=[1,2])
    qml.CZ(wires=[0,2])

We can show that we've acquired the correct state by applying the inverse of our state initialization on qubit 2 and show that it returns to state $\left|0\right>$.

> ***Exercise 3.***
>Complete the following code for the teleportation protocol. (Use the previously defined functions to apply the gates, then use `qml.adjoint` to apply the inverse of the inialization (i.e. the *disentangler*).

The final circuit will look something like this (it's ok if there are more measurements drawn or the barriers are not included):

<div>
<img src="images/qiskit_teleportation_defer.png" width="700">
</div>

In [ ]:
@qml.qnode(dev)
def Teleportation(theta, phi):
    # Your solution here

    return qml.probs(2)

>***Hint:*** `qml.adjoint(f)(params)` takes a gate or function `f` which applies gates and passes it the parameters `params`. Here `f` should be `initialize_psi`, but remember you need to apply the disentangler to qubit `2`.

In [ ]:
np.random.seed(0)
theta = np.pi*np.random.uniform()
phi = np.pi/2*np.random.uniform()

In [ ]:
theta, phi

In [ ]:
# should always be [1, 0] no matter what |psi> is
print(Teleportation(theta,phi))

In [ ]:
dev._circuit.draw('mpl')

------
### Quantum Teleportation on the IBM Quantum Computer

In this section, we will run the teleportation circuit on the IBM Quantum Computer. During the pre-reading, you must have signed up for an [IBMQ account](https://quantum-computing.ibm.com/). On the [IBM Quantum Dashboard](https://quantum-computing.ibm.com/), you should be able to sign-in and access your API token here

![image.png](images/qiskit_api.png)

Simply use the API from the IBM Quantum Dashboard in the device initialization below :

In [ ]:
# Setup the device object to run on real quantum device
# Note - This command can only be run once in a single kernel - if need to re-run, restart the kernel and re-run
dev = qml.device('qiskit.ibmq', wires=3, backend='ibmqx2', shots=2000, ibmqx_token="YOUR_API_KEY_HERE")

In [ ]:
@qml.qnode(dev)
def Teleportation(theta, phi):
    # Your solution here
    
    return qml.probs(2)

In [ ]:
np.random.seed(0)
theta = np.pi*np.random.uniform()
phi = np.pi/2*np.random.uniform()

In [ ]:
theta, phi

In [ ]:
# should always be [1, 0] no matter what |psi> is
print(Teleportation(theta,phi))

In [ ]:
dev._circuit.draw('mpl')

### More Bonus Exercises: Pauli corrections by classical post-processing

As was stated earlier, in the `pennylane` framework, measurement of the circuit has to be performed as the very last step *after* all gates have been applied! This means, it is not possible to implement the Pauli corrections that depend on the measurement outcomes of qubits `0` and `1`.

Instead, we can apply the Pauli corrections *classically*. We measure all three qubits and then post-process each recorded measurement outcome according to the Pauli corrections.

Let's see how this works. Assume we are performing a measurement in the $z$-basis which means that we can differentiate between outcomes $\left|0\right>$ and $\left|1\right>$. The $X$ correction just before measurement swaps the roles of $\left|0\right>$ and $\left|1\right>$. Thus, instead of actually applying an $X$-gate, simply swapping the measured probabilities of getting $\left|0\right>$ and $\left|1\right>$ will do the same job! The $Z$ correction has no effect on the measurement statistics, since it simply adds a phase of $-1$ to $\left|1\right>$ which we can't measure anyways.

> ***Bonus Exercise.*** How can we retroactively apply the $X$ and $Z$ correction when we measure in the $x$-basis, i.e. when we differentiate between outcomes $\left|+\right>$ and $\left|-\right>$?

For an arbitrary single-qubit-state parameterized as $\left|\psi\right> = \cos \frac{\theta}{2}\left|0\right> + e^{i\varphi} \sin\frac{\theta}{2} \left|1\right>$ the expectation values of the Pauli operators are

$$
\begin{aligned}
\langle X \rangle &= \cos \varphi \sin \theta \\
\langle Y \rangle &= \sin \varphi \sin \theta \\
\langle Z \rangle &= \cos \theta \,,
\end{aligned}
$$

i.e. the expectation value of the operators $X,Y,Z$ are the projection of the Bloch vector on the $x,y,z-$axis.

So, if we measure $\langle Z \rangle$, we can compute $\theta$. If we measure a second Pauli operator, we also find $\varphi$. Voilà, we have identified our quantum state!

<div>
<img src="images/bloch_sphere.png" width="300">
</div>

> ***Bonus Exercise.*** Write a python function ` findState(x_exp,z_exp)` that takes expectation values $\langle X \rangle$ and $\langle Z \rangle$ as inputs and returns $\varphi$ and $\theta$. (To keep thing simple, assume that $\varphi \in [0,\pi)$ for our purposes.)

In [ ]:
def findState(x_exp,z_exp):
    # Your solution here
    
    return phi,theta

Below is our implementation of the teleportation circuit that combines all code snippets that you have written so far. There are two circuits. `teleportMeasureX` initializes the input qubit `0` in the state $\left|\psi\right> = \cos \frac{\theta}{2}\left|0\right> + e^{i\varphi} \sin\frac{\theta}{2} \left|1\right>$, implements the teleportation, and measures qubit `2` in the $x$-basis while `teleportMeasureZ` measures the last qubit in the $z$-basis.

In [ ]:
@qml.qnode(dev)
def teleportMeasureX(phi,theta):
    # prepare input state cos(θ/2)|0> + e^(iϕ) sin(θ/2)|1>
    qml.Rot(0,theta,phi, wires=0)
    
    # build the actual circuit
    create_Bell_state()
    Alice_gates()    # rotate to the Bell basis
    
    # rotate the x-basis into the z basis ( |+> --> |0> and |-> --> |1> )
    qml.Hadamard(wires=2)
    
    # return the measurement statistics
    return qml.probs(wires=[0,1,2])

@qml.qnode(dev)
def teleportMeasureZ(phi,theta):
    # prepare input state
    qml.Rot(0,theta,phi, wires=0)
    
    # build the actual circuit
    create_Bell_state()
    Alice_gates()
    
    # return the measurement statistics
    return qml.probs(wires=[0,1,2])

To make things interesting, we randomly choose the parameters of the input state $\left|\psi\right> = \cos \frac{\theta}{2}\left|0\right> + e^{i\varphi} \sin\frac{\theta}{2} \left|1\right>$ as $\varphi,\theta \in [0,\pi)$.

In [ ]:
theta = np.random.rand() * np.pi
phi = np.random.rand() * np.pi

Now we perform the post-processing of the data to apply the Pauli corrections and compute the expectation values $\langle X \rangle$ (`x_exp`) and $\langle Z \rangle $ (`z_exp`). If you are pressured for time, you can simply run the code cell below and understand the steps in the code later.


In [ ]:
binaries = ['000', '001', '010', '011', '100', '101', '110','111']

x_probs = teleportMeasureX(phi, theta) # run the quantum circuit
# the list x_probs will contain the measured probabilities, where the first entry corresponds
# to |000>, the second to |001>, the third to |010>, ...

x_exp = 0 # this variable will contain <X>, we initialize it with 0

for i in np.arange(2**3): # loop through all possible outcomes
    bits = binaries[i]
    outcome = int(bits[2]) # get the state of the last qubit
    eigenvalue = (-2*outcome + 1) # the eigenvalue is 1 if the qubit was measured to be 0. It is -1 if the qubit was measured 1
    if bits[0]=='1':
        eigenvalue = (-eigenvalue) # Apply the Pauli-Z correction, i.e. flip the eigenvalue if qubit 0 was measured 1
        # The Pauli-X correction has no effect since we are measuring in the X-basis
    x_exp += eigenvalue * x_probs[i] # compute the expectation value as <X> = \sum_i probs_i * eigenvalue_i


z_probs = teleportMeasureZ(phi, theta)
z_exp = 0

for i in np.arange(2**3): # loop through all possible outcomes
    bits = binaries[i]
    outcome = int(bits[2])
    eigenvalue = (-2*outcome + 1)
    if bits[1]=='1':
        eigenvalue = (-eigenvalue) # Apply the Pauli-X correction, i.e. flip the eigenvalue if qubit 1 was measured 1
        # The Pauli-Z correction has no effect since we are measuring in the Z-basis
    z_exp += eigenvalue * z_probs[i]

> ***Bonus Exercise.*** Scrap the above code cell and program your own data processing routine from scratch.

Finally, having computed $\langle X\rangle$ and $\langle Z\rangle$ we can use our function `findState` to compute the angles of our Bloch vector:

In [ ]:
phi_measured, theta_measured = findState(x_exp, z_exp)
print('Measured angles: ϕ=', phi_measured, ', θ=', theta_measured)

Let's compare this to the actual parameters of the initial state. Hopefully they agree with our measurement!

In [ ]:
print('Initial state: ϕ=', phi, ', θ=', theta)

### References
1. M. Nielsen and I. Chuang, Quantum Computation and Quantum Information (2000)
2. `qiskit` [textbook](https://qiskit.org/textbook) 

# GHZ states
------
[author]: # (Written by Giovanni Iannelli and Paolo Stornati, July 2021. Please email me us at <giovanni.iannelli@desy.de> or <paolo.stornati@desy.de> in case of errors.)

In [1]:
import pennylane as qml
from pennylane import numpy as np

In yesterdays's homework, we learned how to construct two-qubit entangled states (_Bell states_) with PennyLane by applying a Hadamard gate followed by a CNOT. This worksheet will build upon that knowledge and extend the ideas of entanglement to three qubits.
 

While the canonical example of an entanglement is a 2-qubit system, this is just the tip of the iceberg. Systems with more than 2 qubits can contain varying degrees of entanglement. For example, in a 3-qubit system, it may be the case that  
- None of the qubits are entangled and the qubits are in a product state (e.g $|000\rangle$, $|010\rangle$ ...)
 - a pair of two are entangled and the third is separate, e.g. $\frac{1}{\sqrt{2}} \left( |00\rangle + |11 \rangle \right) |0\rangle$ 
 - all three are fully entangled. 
 
The GHZ state is an example of the third case, and is in a sense the logical extension of the Bell state. A 3-qubit GHZ state is defined as
$$
|\hbox{GHZ} \rangle_3 = \frac{1}{\sqrt{2}} \left( |000\rangle + |111 \rangle \right) 
$$
GHZ states are named after the researchers who first studied them: [Greenberger, Horne, and Zeilinger](https://arxiv.org/abs/0712.0921).

Let's make one!

> **Exercise 1.1:** Starting from $|00\rangle$, we built the Bell state in the following way:
$$C_{NOT}^{0,1}H^0|00\rangle = C_{NOT}^{0,1}\frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)|0\rangle =C_{NOT}^{0,1}\frac{1}{\sqrt{2}}(|00\rangle + |10\rangle)=\frac{1}{\sqrt{2}}(|00\rangle+|11\rangle).$$
How is then possible to build a GHZ state starting from $|000\rangle$? (Hint: Start with the same two gate operations used for the Bell state, i.e. start applying $C_{NOT}^{0,1}H^0$ to $|000\rangle$. The solution is not unique.)

> **Exercise 1.2:** Implement your answer to exercise 1.1 in PennyLane. After that, run the test and verify that the state has 50% probability of being observed in state $|000\rangle$, 50% in $|111\rangle$ and 0% in all other states.

In [ ]:
# This is a helper function to output the basis state probabilities
# Run this cell, but no need to change anything here
def pretty_print_probabilities(num_qubits, probs):
    print("Basis state probabilities: ")
    format_string = f'0{num_qubits}b'
    for state_idx in range(2**num_qubits):
        print(f"|{format(state_idx, format_string)}>: {probs[state_idx]:.3f}")

In [ ]:
# Sets up a device with 3 qubits
dev = qml.device('default.qubit', wires=3, shots=None)

@qml.qnode(dev)
def GHZ_3(): 
    # Hint: add gates to the circuit with qml.Hadamard(...) and qml.CNOT(...)
    # Your solution here
    
    return qml.probs(wires=range(dev.num_wires))

In [ ]:
# Run the circuit and print results
basis_state_probabilities = GHZ_3()
pretty_print_probabilities(dev.num_wires, basis_state_probabilities)

> **Exercise 1.3:** Let's check whether the three qubits in the GHZ state are fully entangled. Before any measure, every qubit has 50% probability of being observed in state 0 or 1. If one of the three qubits is observed in state 1, what's the probability of the other two qubits being in state 1 as well?

The other two qubits will be in state 1 as well in 100% of cases. Measuring one qubit then modifies the state of the other two qubits, hence all three qubits are entangled.

> **Exercise 1.4:** Let's run the same circuit in a real IBMQ device using 100 shots. Do we observe only two outcomes as before? Are the conclusions drawn in the previous exercises still valid?

In [ ]:
dev = qml.device('qiskit.ibmq', wires=3, backend='ibmqx2', shots=100, ibmqx_token="YOUR_API_KEY_HERE")

@qml.qnode(dev)
def GHZ_3_device():
    # Your solution here
    
    return qml.probs(wires=range(dev.num_wires))

In [ ]:
# Run the circuit and print results
# Using the real device might take a long time
basis_state_probabilities = GHZ_3_device()
pretty_print_probabilities(dev.num_wires, basis_state_probabilities)

The system is still observed most of the times in the states 000 or 111, each of them with similar probability. However, they are not the only states observed. Therefore, observing just one qubit is not sufficient anymore to infer the state of the other two. This alteration of results is due to gate errors and quantum decoherence.

> **Bonus 1.5:** As a point of interest, GHZ states can be generalized to $n$ qubits, 
 $$
 |\hbox{GHZ} \rangle_n = \frac{1}{\sqrt{2}} \left( |0\rangle^{\otimes n} + |1 \rangle^{\otimes n} \right),
 $$
> where the notation $|0\rangle^{\otimes n}$ means $n$ tensor product copies of $|0\rangle$.
> Generalize the construction of the GHZ state to arbitrary $n$.

In [ ]:
# Change n to whatever you like
n = 4

dev = qml.device('default.qubit', wires=n, shots=None)

@qml.qnode(dev)
def GHZ_n():
    # Your solution here
    
    return qml.probs(wires=range(dev.num_wires))

In [ ]:
# Run the circuit and print results
basis_state_probabilities = GHZ_n()
pretty_print_probabilities(dev.num_wires, basis_state_probabilities)

In [ ]:
print("With the second method: ")
basis_state_probabilities_2 = GHZ_n_version_2()
pretty_print_probabilities(dev.num_wires, basis_state_probabilities_2)

> **Bonus 1.5:** We are often interested in the resource requirements of quantum circuits, especially in the present era where operations are noisy, and coherence times of qubits are low. Common metrics of circuit quality are width (number of qubits), depth (number of sequential operations that cannot be parallelized), and the number of specific types of gates, especially 2-qubit gates. For the `GHZ_n` circuit you wrote above, what are the width, depth, and gate counts in terms of $n$?